# Creating a basic RAG





## Getting the embeddings model

Eval benchamrk for [polish sentences](https://github.com/sdadas/polish-sentence-evaluation).

We shall experiment with the following:
- `distiluse-base-multilingual-cased-v2`
- `xlm-r-distilroberta-base-paraphrase-v1`
- `xlm-r-bert-base-nli-stsb-mean-tokens`
- `distilbert-multilingual-nli-stsb-quora-ranking`

First, we will get the embeddings model `xlm-r-distilroberta-base-paraphrase-v1`. All come pretrained from [Sentence Transformers](https://www.sbert.net/docs/pretrained_models.html).


In [4]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

/Users/piotr/projects/llm-zoomcamp-project/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/piotr/projects/llm-zoomcamp-project/.venv/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [5]:
model_name = 'xlm-r-distilroberta-base-paraphrase-v1'
embedding_model = SentenceTransformer(model_name)

/Users/piotr/projects/llm-zoomcamp-project/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Python(25757) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [34]:

user_question = "Jak mogę usunąć swoje konto?"

In [35]:
v = embedding_model.encode(user_question)

## Prepare the documents

Now we will create the embeddings for the documents.




In [1]:
import requests 
import json

with open('../data/documents-olx-2024_08_25-with-ids.json', 'r',  encoding='utf8') as f_in:
     documents = json.load(f_in)

In [2]:
len(documents)

90


## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

- Create a list `embeddings` 
- Iterate over each document 
- `qa_text = f'{question} {text}'`
- compute the embedding for `qa_text`, append to `embeddings`
- At the end, let `X = np.array(embeddings)` (`import numpy as np`) 

What's the shape of X? (`X.shape`). Include the parantheses. 




In [3]:
import numpy as np

In [6]:
#created the dense vector using the pre-trained model
embeddings = []
for doc in documents:
    # Transforming the title into an embedding using the model
    qa_text = f'{doc["question"]} {doc["text"]}'
    embeddings.append(embedding_model.encode(qa_text).tolist())

In [7]:
X = np.array(embeddings)

In [8]:
X.shape

(90, 768)

## Vector search

We can now compute the similarity between a query vector and all the embeddings.


In [42]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Usunięcie konta OLX spowoduje, że jego odzyskanie nie będzie już możliwe. Wszystkie Twoje dane zostaną skasowane.\nKrok 1. Zaloguj się na konto OLX.\nKrok 2. Jeśli korzystasz z:\nstrony internetowej OLX — kliknij Twoje konto → Ustawienia → Zarządzanie kontem.\naplikacji mobilnej OLX — kliknij Konto → Ustawienia.\nKrok 3. Kliknij przycisk Usuń konto → Wyślij e-mail.\nKrok 4. Przejdź na adres e-mail powiązany z kontem OLX i potwierdź chęć usunięcia konta.\nZmiana e-mail/danych kontaktowych\nZmiana adresu e-mail\nZmiana danych kontaktowych',
  'section': 'Konto',
  'question': 'Jak usunąć konto?',
  'id': '4dcf8b35'},
 {'text': 'Użytkownik może posiadać tylko jedno Konto w Serwisie. Powyższa zasada nie dotyczy przypadków, gdy:\nUżytkownik posiada jedno Konto służące do celów prywatnych oraz, z zastrzeżeniem podpunktu b poniżej, Konta służące do celów związanych z prowadzoną przez niego działalnością gospodarczą;\nUżytkownik wykorzystuje różne Konta w zakresie prowadzonej przez 

## Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will
use the hitrate metric for evaluation.

** TO DO **

Get ground truth data from the `data` folder.

** TO DO **

Calculate hitrate for the search engine.

** TO DO **

Calculate the mrr for the search engine.

In [20]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = search_engine.search(embedding_model.encode(q['question']), num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████| 1830/1830 [01:07<00:00, 27.28it/s]


In [22]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [23]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [24]:
hit_rate(relevance_total), mrr(relevance_total)

(0.9398907103825137, 0.8516484517304189)

## Indexing with Elasticsearch

Now let's index these documents with elasticsearch

* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)

After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?



In [9]:
from elasticsearch import Elasticsearch
from datetime import datetime

es_client = Elasticsearch('http://localhost:9200') 

In [10]:
index_name_prefix = 'documents_olx'
current_time = datetime.now().strftime("%Y%m%d_%M%S")
index_name = f"{index_name_prefix}_{current_time}"
print("index name:", index_name)

index name: documents_olx_20240907_0504


In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}



es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'documents_olx_20240907_0504'})

In [12]:
len(documents)

90

In [13]:
for doc, embedding in tqdm(zip(documents, embeddings)):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = embedding_model.encode(question)
    doc['text_vector'] = embedding_model.encode(text)
    doc['question_text_vector'] = embedding_model.encode(qt)

0it [00:00, ?it/s]

90it [00:42,  2.13it/s]


In [50]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'id', 'question_vector', 'text_vector', 'question_text_vector'])

In [54]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 90/90 [00:02<00:00, 43.24it/s]


In [55]:
query = {
    "field" : "question_text_vector",
    "query_vector" :  v,
    "k" : 5,
    "num_candidates" : 10000, 
}

res = es_client.search(index=index_name, 
                       knn=query,
                       source=["text","section","question","course", "id"])
res["hits"]["hits"]

[{'_index': 'documents_olx_20240829_5014',
  '_id': 'PJv4nZEBcxG-ujiI-xmj',
  '_score': 0.7219515,
  '_source': {'question': 'Jak usunąć konto?',
   'section': 'Konto',
   'text': 'Usunięcie konta OLX spowoduje, że jego odzyskanie nie będzie już możliwe. Wszystkie Twoje dane zostaną skasowane.\nKrok 1. Zaloguj się na konto OLX.\nKrok 2. Jeśli korzystasz z:\nstrony internetowej OLX — kliknij Twoje konto → Ustawienia → Zarządzanie kontem.\naplikacji mobilnej OLX — kliknij Konto → Ustawienia.\nKrok 3. Kliknij przycisk Usuń konto → Wyślij e-mail.\nKrok 4. Przejdź na adres e-mail powiązany z kontem OLX i potwierdź chęć usunięcia konta.\nZmiana e-mail/danych kontaktowych\nZmiana adresu e-mail\nZmiana danych kontaktowych',
   'id': '4dcf8b35'}},
 {'_index': 'documents_olx_20240829_5014',
  '_id': 'c5v4nZEBcxG-ujiI_hnJ',
  '_score': 0.6761112,
  '_source': {'question': 'Wiele kont użytkownika',
   'section': 'Konta',
   'text': 'Użytkownik może posiadać tylko jedno Konto w Serwisie. Powyższa z

In [56]:
def elastic_search_knn(field, vector, index_name):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [57]:
search_results = elastic_search_knn("question_text_vector", v, index_name)

In [58]:
search_results

[{'question': 'Jak usunąć konto?',
  'section': 'Konto',
  'text': 'Usunięcie konta OLX spowoduje, że jego odzyskanie nie będzie już możliwe. Wszystkie Twoje dane zostaną skasowane.\nKrok 1. Zaloguj się na konto OLX.\nKrok 2. Jeśli korzystasz z:\nstrony internetowej OLX — kliknij Twoje konto → Ustawienia → Zarządzanie kontem.\naplikacji mobilnej OLX — kliknij Konto → Ustawienia.\nKrok 3. Kliknij przycisk Usuń konto → Wyślij e-mail.\nKrok 4. Przejdź na adres e-mail powiązany z kontem OLX i potwierdź chęć usunięcia konta.\nZmiana e-mail/danych kontaktowych\nZmiana adresu e-mail\nZmiana danych kontaktowych',
  'id': '4dcf8b35'},
 {'question': 'Wiele kont użytkownika',
  'section': 'Konta',
  'text': 'Użytkownik może posiadać tylko jedno Konto w Serwisie. Powyższa zasada nie dotyczy przypadków, gdy:\nUżytkownik posiada jedno Konto służące do celów prywatnych oraz, z zastrzeżeniem podpunktu b poniżej, Konta służące do celów związanych z prowadzoną przez niego działalnością gospodarczą;\nUży

## Prompt building

In [59]:
def build_prompt(query, search_results):
    prompt_template = """
Jesteś pracownikiem działu obsługi klienta firmy OLX. 
Odpowiedz na PYTANIE klienta bazując na KONTEKST pochodzący z bazy danych częstych pytań i odpowiedzi oraz regulaminu serwisu.
Bazuj jedynie na wiedzy z KONTEKST kiedy odpowiadasz na PYTANIE.

PYTANIE: {question}

KONTEKST: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [67]:
prompt = build_prompt(user_question, search_results)

In [69]:
print(prompt)

Jesteś pracownikiem działu obsługi klienta firmy OLX. 
Odpowiedz na PYTANIE klienta bazując na KONTEKST pochodzący z bazy danych częstych pytań i odpowiedzi oraz regulaminu serwisu.
Bazuj jedynie na wiedzy z KONTEKST kiedy odpowiadasz na PYTANIE.

PYTANIE: Jak mogę usunąć swoje konto?

KONTEKST: 
section: Konto
question: Jak usunąć konto?
answer: Usunięcie konta OLX spowoduje, że jego odzyskanie nie będzie już możliwe. Wszystkie Twoje dane zostaną skasowane.
Krok 1. Zaloguj się na konto OLX.
Krok 2. Jeśli korzystasz z:
strony internetowej OLX — kliknij Twoje konto → Ustawienia → Zarządzanie kontem.
aplikacji mobilnej OLX — kliknij Konto → Ustawienia.
Krok 3. Kliknij przycisk Usuń konto → Wyślij e-mail.
Krok 4. Przejdź na adres e-mail powiązany z kontem OLX i potwierdź chęć usunięcia konta.
Zmiana e-mail/danych kontaktowych
Zmiana adresu e-mail
Zmiana danych kontaktowych

section: Konta
question: Wiele kont użytkownika
answer: Użytkownik może posiadać tylko jedno Konto w Serwisie. Powyż

In [61]:
from openai import OpenAI

In [62]:
client = OpenAI()

In [70]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [71]:
llm_response = llm(prompt)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [73]:
print(llm_response)

Aby usunąć swoje konto OLX, postępuj zgodnie z poniższymi krokami:

1. Zaloguj się na swoje konto OLX.
2. Jeśli korzystasz ze strony internetowej OLX, przejdź do sekcji "Twoje konto", następnie "Ustawienia" i wybierz "Zarządzanie kontem". W aplikacji mobilnej OLX, kliknij "Konto", a następnie "Ustawienia".
3. Kliknij przycisk "Usuń konto" i wyślij e-mail.
4. Przejdź na adres e-mail powiązany z Twoim kontem OLX i potwierdź chęć usunięcia konta.

Pamiętaj, że usunięcie konta OLX spowoduje trwałe usunięcie wszystkich Twoich danych i procesu nie będzie można cofnąć. Jeśli masz zgromadzone środki na Portfelu OLX, wcześniej musisz skontaktować się z Grupą OLX za pomocą formularza kontaktowego w celu wypłaty tych środków. 

Jeśli zdecydujesz się usunąć konto, umowa usługi Konta zostanie rozwiązana, co również spowoduje wygaśnięcie pozostałych umów dotyczących usług świadczonych w ramach Serwisu.


In [76]:
def rag(query):
    encoded_query = embedding_model.encode(query)
    search_results = elastic_search_knn("question_text_vector", encoded_query, index_name)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [78]:
rag("Jak zapłacić za ogłoszenie?")

'Aby zapłacić za ogłoszenie na OLX, możesz skorzystać z różnych metod płatności dostępnych podczas publikowania ogłoszeń. Proces ten zazwyczaj obejmuje dodanie ogłoszenia, po czym pojawią się dostępne opcje płatności, takie jak:\n\n1. **BLIK** — Podajesz 6-cyfrowy kod z aplikacji swojego banku.\n2. **Przelew online** — Wybierasz swój bank z listy i logujesz się do swojego konta bankowego, aby wykonać przelew.\n\nWybór konkretnej metody płatności zależy od dostępnych opcji widocznych podczas finalizowania transakcji. Upewnij się również, że Twoje konto bankowe obsługuje BLIK, jeśli zdecydujesz się na tę metodę.\n\nJeśli masz jakiekolwiek trudności lub pytania dotyczące płatności, możesz także skontaktować się z naszym działem obsługi klienta za pomocą formularza kontaktowego dostępnego na stronie OLX.'

In [79]:
index_name

'documents_olx_20240821_4322'

In [14]:
import psycopg2

def get_db_connection():
    return psycopg2.connect(
        host="postgres",
        database="your_database",
        user="your_user",
        password="your_password"
    )
